In [50]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy as st 


In [51]:
# Study data files 
mouse_metadata = "data/Mouse_metadata.csv"
study_results = "data/Study_results.csv"

In [52]:
# Read the mouse data and the study results 
mouse_metadata = pd.read_csv(mouse_metadata)
study_results = pd.read_csv(study_results)

# Combine the data into a single dataset 

pymaceuticals_data = pd.merge(mouse_metadata, study_results)

pymaceuticals_data.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [53]:
# Display the number of unique mice IDs in the data
pymaceuticals_data.groupby['Mouse ID'].count()

TypeError: 'method' object is not subscriptable